In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/keywords.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links.csv


In [101]:
meta = pd.read_csv("/kaggle/input/the-movies-dataset/movies_metadata.csv")
meta = meta[["id","original_title","original_language","genres"]]
meta = meta.rename(columns={"id":"movieId", "original_title":"title","original_language":"language"})
meta = meta.loc[meta['language'] == "en",:]
meta.movieId = pd.to_numeric(meta.movieId)
meta

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,movieId,title,language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"
...,...,...,...,...
45459,222848,Caged Heat 3000,en,"[{'id': 878, 'name': 'Science Fiction'}]"
45460,30840,Robin Hood,en,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name..."
45463,67758,Betrayal,en,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam..."
45464,227506,Satana likuyushchiy,en,[]


In [102]:
def str_to_set(genre):
    myset = set()
    for d in eval(genre):
        myset.add(d['name'])
    return myset
meta.genres = meta.genres.apply(str_to_set)


In [103]:
keywords = pd.read_csv("/kaggle/input/the-movies-dataset/keywords.csv")

In [104]:
keywords['keywords'] = keywords['keywords'].apply(str_to_set)

In [105]:
keywords = keywords.rename(columns={'id':'movieId'})
keywords

,movieId,keywords
0,862,"{rivalry, boy next door, boy, jealousy, friend..."
1,8844,"{board game, disappearance, giant insect, recl..."
2,15602,"{duringcreditsstinger, fishing, old men, best ..."
3,31357,"{based on novel, single mother, chick flick, i..."
4,11862,"{pregnancy, gynecologist, baby, daughter, midl..."
...,...,...
46414,439050,{tragic love}
46415,111109,"{pinoy, play, artist}"
46416,67758,{}
46417,227506,{}


In [106]:
meta = pd.merge(meta, keywords, on='movieId', how='inner')
meta

,movieId,title,language,genres,keywords
0,862,Toy Story,en,"{Comedy, Animation, Family}","{rivalry, boy next door, boy, jealousy, friend..."
1,8844,Jumanji,en,"{Family, Adventure, Fantasy}","{board game, disappearance, giant insect, recl..."
2,15602,Grumpier Old Men,en,"{Romance, Comedy}","{duringcreditsstinger, fishing, old men, best ..."
3,31357,Waiting to Exhale,en,"{Romance, Comedy, Drama}","{based on novel, single mother, chick flick, i..."
4,11862,Father of the Bride Part II,en,{Comedy},"{pregnancy, gynecologist, baby, daughter, midl..."
...,...,...,...,...,...
32847,222848,Caged Heat 3000,en,{Science Fiction},{}
32848,30840,Robin Hood,en,"{Romance, Action, Drama}",{}
32849,67758,Betrayal,en,"{Action, Drama, Thriller}",{}
32850,227506,Satana likuyushchiy,en,{},{}


In [107]:
dk = meta.loc[meta['title'] == "The Dark Knight"].iloc[0]
dkr = meta.loc[meta['title'] == "The Dark Knight Rises"].iloc[0]
pd.concat([dk,dkr],axis=1).T

,movieId,title,language,genres,keywords
10278,155,The Dark Knight,en,"{Action, Crime, Drama, Thriller}","{superhero, batman, based on comic, district a..."
14315,49026,The Dark Knight Rises,en,"{Action, Crime, Drama, Thriller}","{superhero, time bomb, destruction, batman, vi..."


In [108]:
def jaccard_similarity(s1,s2):
    if len(s1|s2) == 0:
        return 0
    return len(s1&s2)/len(s1|s2)

In [111]:
# dk.genres | dk.keywords
jaccard_similarity(dk.genres | dk.keywords, dkr.genres | dkr.keywords)

0.37142857142857144

In [110]:
dk

movieId                                                   155
title                                         The Dark Knight
language                                                   en
genres                       {Action, Crime, Drama, Thriller}
keywords    {superhero, batman, based on comic, district a...
Name: 10278, dtype: object

In [112]:
ratings = pd.read_csv("/kaggle/input/the-movies-dataset/ratings_small.csv")
ratings

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
...,...,...,...,...
99999,671,6268,2.5,1065579370
100000,671,6269,4.0,1065149201
100001,671,6365,4.0,1070940363
100002,671,6385,2.5,1070979663


In [113]:
ratings = pd.merge(ratings, meta[['movieId','title']], on='movieId', how="inner")
ratings

,userId,movieId,rating,timestamp,title
0,1,1371,2.5,1260759135,Rocky III
1,4,1371,4.0,949810302,Rocky III
2,7,1371,3.0,851869160,Rocky III
3,19,1371,4.0,855193404,Rocky III
4,21,1371,3.0,853852263,Rocky III
...,...,...,...,...,...
33295,652,127728,5.0,1439586990,8:46
33296,652,129009,4.0,1442690827,Love Is a Ball
33297,653,2103,3.0,948161066,Solaris
33298,659,167,4.0,836137550,K-PAX


In [114]:
matrix = matrix = ratings.pivot_table(index="userId", columns="title", values="rating")
matrix

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [116]:
def pearson_similarity(a,b):
    a = a - a.mean()
    b = b - b.mean()
    denom = ((np.sum(a ** 2)) * (np.sum(b ** 2))) ** 0.5
    if denom != 0:
        return np.sum(a * b) / denom
    else:
        return 0

In [117]:
dk_rating = matrix["The Dark Knight"]
pn_rating = matrix["Prom Night"]
pearson_similarity(dk_rating, pn_rating)

0.773565934694095

In [118]:
def find_similar_movies (input_title , matrix, n, alpha):
    input_meta = meta.loc[ meta[ 'title'] == input_title].iloc[ 0]
    input_set = input_meta.genres | input_meta.keywords

    result = []

    for this_title in matrix.columns:
        if this_title == input_title:
            continue
        this_meta = meta.loc[ meta[ 'title'] == this_title].iloc[ 0]
        this_set = this_meta.genres | this_meta.keywords

        pearson = pearson_similarity(matrix[this_title], matrix[input_title])
        jaccard = jaccard_similarity(this_set, input_set)

        score = alpha * pearson + ( 1-alpha) * jaccard
        result.append( (this_title, pearson, jaccard, score) )

    result.sort(key= lambda r: r[3], reverse= True)

    return result[:n]

In [119]:
result = find_similar_movies('The Dark Knight', matrix, 10, 0.3)
pd.DataFrame(result, columns = ['title','pearson','jaccard','score'])

,title,pearson,jaccard,score
0,Wild Wild West,0.773566,0.032258,0.254650
1,Prom Night,0.773566,0.022222,0.247625
2,Batman Begins,0.005015,0.292683,0.206383
3,Yamakasi - Les samouraïs des temps modernes,0.377145,0.125000,0.200643
4,Blue Thunder,0.326617,0.133333,0.191318
5,Midnight in the Garden of Good and Evil,0.373841,0.111111,0.189930
6,Topaz,0.377145,0.103448,0.185557
7,Big Bad Mama,0.344649,0.107143,0.178395
8,Sneakers,0.415830,0.068966,0.173025
9,The Enforcer,0.326617,0.103448,0.170399
